# In this notebook we will try to predict the power generation for the next 7 days on the solar power generation dataset

# Approach : we will first create ground truth for the next seven days using the FB Prophet model , and then we will let our LSTM based model
# train on this data and make predictions on next 7 days

# # I had to create this notebook on a short notice so please bear with me if you feel the code is not clean, I will soon upload a cleaner version of the notebook.Also I have combined the work from 3 different notebook, if some code is broken or you find completely detached code segment please let me know in comments

# Disclaimer : I have taken some of the data preprocessing ideas from the other notebooks which can be found of kaggle
# please upvote if you like this notebook

In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [ ]:
dfGeneration_1=pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Generation_Data.csv')
dfWeatherSensor_1=pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_1_Weather_Sensor_Data.csv')
dfGeneration_2=pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Generation_Data.csv')
dfWeatherSensor_2=pd.read_csv('/kaggle/input/solar-power-generation-data/Plant_2_Weather_Sensor_Data.csv')

In [ ]:
dfGeneration_1.describe()

In [ ]:
dfGeneration_1['DATE_TIME'] = pd.to_datetime(dfGeneration_1['DATE_TIME'], format ='%d-%m-%Y %H:%M')
dfGeneration_1['DATE'] = dfGeneration_1['DATE_TIME'].dt.date
dfGeneration_1['DATE'] = pd.to_datetime(dfGeneration_1['DATE'],format = '%Y-%m-%d')
dfGeneration_1['TIME'] = dfGeneration_1['DATE_TIME'].dt.time
dfGeneration_1['TIME'] = pd.to_datetime(dfGeneration_1['TIME'], format = '%H:%M:%S')
dfGeneration_1['HOUR'] = pd.to_datetime(dfGeneration_1['TIME'],format='%H:%M:%S').dt.hour
dfGeneration_1['MINUTES'] = pd.to_datetime(dfGeneration_1['TIME'],format='%H:%M:%S').dt.minute
dfGeneration_1.info()

dfGeneration_2['DATE_TIME'] = pd.to_datetime(dfGeneration_2['DATE_TIME'], format ='%Y-%m-%d %H:%M')
dfGeneration_2['DATE'] = dfGeneration_2['DATE_TIME'].dt.date
dfGeneration_2['DATE'] = pd.to_datetime(dfGeneration_2['DATE'],format = '%Y-%m-%d')
dfGeneration_2['TIME'] = dfGeneration_2['DATE_TIME'].dt.time
dfGeneration_2['TIME'] = pd.to_datetime(dfGeneration_2['TIME'], format = '%H:%M:%S')
dfGeneration_2['HOUR'] = pd.to_datetime(dfGeneration_2['TIME'],format='%H:%M:%S').dt.hour
dfGeneration_2['MINUTES'] = pd.to_datetime(dfGeneration_2['TIME'],format='%H:%M:%S').dt.minute
dfGeneration_2.info()

dfWeatherSensor_1['DATE_TIME'] = pd.to_datetime(dfWeatherSensor_1['DATE_TIME'], format ='%Y-%m-%d %H:%M')
dfWeatherSensor_1['DATE'] = dfWeatherSensor_1['DATE_TIME'].dt.date
dfWeatherSensor_1['DATE'] = pd.to_datetime(dfWeatherSensor_1['DATE'],format = '%Y-%m-%d')
dfWeatherSensor_1['TIME'] = dfWeatherSensor_1['DATE_TIME'].dt.time
dfWeatherSensor_1['TIME'] = pd.to_datetime(dfWeatherSensor_1['TIME'], format = '%H:%M:%S')
dfWeatherSensor_1['HOUR'] = pd.to_datetime(dfWeatherSensor_1['TIME'],format='%H:%M:%S').dt.hour
dfWeatherSensor_1['MINUTES'] = pd.to_datetime(dfWeatherSensor_1['TIME'],format='%H:%M:%S').dt.minute
dfWeatherSensor_1.info()

dfWeatherSensor_2['DATE_TIME'] = pd.to_datetime(dfWeatherSensor_2['DATE_TIME'], format ='%Y-%m-%d %H:%M')
dfWeatherSensor_2['DATE'] = dfWeatherSensor_2['DATE_TIME'].dt.date
dfWeatherSensor_2['DATE'] = pd.to_datetime(dfWeatherSensor_2['DATE'],format = '%Y-%m-%d')
dfWeatherSensor_2['TIME'] = dfWeatherSensor_2['DATE_TIME'].dt.time
dfWeatherSensor_2['TIME'] = pd.to_datetime(dfWeatherSensor_2['TIME'], format = '%H:%M:%S')
dfWeatherSensor_2['HOUR'] = pd.to_datetime(dfWeatherSensor_2['TIME'],format='%H:%M:%S').dt.hour
dfWeatherSensor_2['MINUTES'] = pd.to_datetime(dfWeatherSensor_2['TIME'],format='%H:%M:%S').dt.minute

In [ ]:
# let's check for any missing values

dfGeneration_1.isna().sum()

# as we can see there are no missing values in the data

In [ ]:
# let's look at some of the power generation trends

dfGeneration_1.DC_POWER.value_counts()

In [ ]:
# As we can see most of the time stamps show power generated is zero which makes it difficult for us to train a model 

In [ ]:
dfGeneration_1.AC_POWER.value_counts(normalize=True)

In [ ]:
# same thing can be seen in Ac power generation 46% of the entries show no power generation

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.plot(dfGeneration_1.groupby('DATE')['DAILY_YIELD'].sum())
plt.title('Daily Yield Plant 1')
plt.xlabel('Date')
plt.ylabel('Daily Yield')


In [ ]:
plt.figure(figsize=(15,8))
plt.plot(dfGeneration_2.groupby('DATE')['DAILY_YIELD'].sum(),'red')
plt.title('Daily Yield Plant 2')
plt.xlabel('Date')
plt.ylabel('Daily Yield')

In [ ]:
plt.figure(figsize=(15,8))
plt.title('AC_POWER Plant 1 VS Plant 2')
plt.plot(dfGeneration_2.groupby('DATE')['AC_POWER'].sum(),'red')
plt.plot(dfGeneration_1.groupby('DATE')['AC_POWER'].sum(),'blue')
plt.xlabel('Date')
plt.ylabel('AC_POWER')

In [ ]:
plt.figure(figsize=(15,8))
plt.title('Daily Yield Plant 1 VS Plant 2')
plt.plot(dfGeneration_2.groupby('DATE')['DAILY_YIELD'].sum(),'red')
plt.plot(dfGeneration_1.groupby('DATE')['DAILY_YIELD'].sum(),'blue')
plt.xlabel('Date')
plt.ylabel('Daily Yield')

In [ ]:
plant2=dfGeneration_2.groupby('DATE')['DC_POWER'].sum().values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler(feature_range=(-1,1))

plant2=np.expand_dims(plant2,-1)
plant2=scaler.fit_transform(plant2)

In [ ]:
merged=pd.merge(dfGeneration_1,dfWeatherSensor_1,on='DATE')

In [ ]:
import seaborn as sns

plt.figure(figsize=(18,8))
sns.heatmap(merged.corr(),annot=True)

In [ ]:
df_plant1 = pd.merge(dfGeneration_1, dfWeatherSensor_1, on='DATE_TIME', how='left')
plt.figure(figsize=(20,8))
plt.title("Ambient Temperature vs DC Power Plant 1")
plt.scatter(df_plant1['AMBIENT_TEMPERATURE'], df_plant1['DC_POWER'], c='c', alpha=0.9,marker='.')
plt.xlabel("Ambient Temperature")
plt.ylabel("DC Power")
plt.grid()
plt.show()

In [ ]:
df_plant1 = pd.merge(dfGeneration_1, dfWeatherSensor_1, on='DATE_TIME', how='left')
plt.figure(figsize=(20,8))
plt.title("MODULE_TEMPERATURE vs DC Power Plant 1")
plt.scatter(df_plant1['MODULE_TEMPERATURE'], df_plant1['DC_POWER'], c='c', alpha=0.9,marker='.')
plt.xlabel("MODULE Temperature")
plt.ylabel("DC Power")
plt.grid()
plt.show()

# let's check if the temeprature outside and temperature of the panel makes any difference

In [ ]:
df_plant1['temp_diff']=df_plant1['MODULE_TEMPERATURE']-df_plant1['AMBIENT_TEMPERATURE']

In [ ]:
df_plant1.sample(5)

In [ ]:
plt.scatter(df_plant1['temp_diff'],df_plant1['DC_POWER'])

# we can see there is a strong corerelation between light and power generated

In [ ]:
# let's check the effect of the light on the panels

In [ ]:
plt.scatter(df_plant1['IRRADIATION'],df_plant1['DC_POWER'])

In [ ]:
plant2=plant2.flatten()

In [ ]:
from math import sqrt


In [ ]:
# As we can see in the graph the Prophet is not able to fit the data very well
import fbprophet as fb

ds = dfGeneration_1['DATE'].unique()
y = dfGeneration_1.groupby('DATE')['DC_POWER'].sum()
df = pd.DataFrame({'ds':ds, 'y':y})
op1 = fb.Prophet(changepoint_prior_scale = .25)
op1.fit(df)
prediction1 = op1.make_future_dataframe(periods = 7, freq = 'D')
prediction1 = op1.predict(prediction1)
op1.plot(prediction1, xlabel = 'DATE', ylabel = 'DC POWER (x10^6)', figsize = (20,10))
plt.title("DC POWER FORECAST FOR PLANT 1")
plt.show()

In [ ]:
values=prediction1.iloc[:,-1]
values=np.expand_dims(values,-1)
values=scaler.inverse_transform(values)
values

In [ ]:
# Now we have ground truth available to us ,

# let's start with our model


from keras.layers import Bidirectional,LSTM,Dense,Flatten,Conv1D,MaxPooling1D,Dropout,RepeatVector
from keras.models import Sequential
from keras.callbacks import EarlyStopping,ReduceLROnPlateau

early_stop=EarlyStopping(monitor='loss',patience=5)


model=Sequential([Conv1D(100,kernel_size=3,input_shape=(7,1),activation='relu'),
                  
                 Flatten(),
                  RepeatVector(7),
                 LSTM(128,activation='relu',return_sequences=True),
                 LSTM(128,activation='relu',return_sequences=True),
                  Bidirectional(LSTM(64,activation='relu')),
                 Dense(128,activation='relu'),
                 Dense(1)])

model.compile(optimizer='adam',loss='mse',metrics=['mse'])

In [ ]:
values=prediction1.iloc[:,-1]
values=np.expand_dims(values,-1)
values=scaler.inverse_transform(values)
values

In [ ]:
values=scaler.fit_transform(values)

In [ ]:
steps=7
inp1=[]
out1=[]

for i in range(len(values)-steps):
    inp1.append(values[i:i+steps])
    out1.append(values[i+steps])
    
inp1=np.asanyarray(inp1)
out1=np.asanyarray(out1)
x_train1=inp1[:27,:,:]
x_test1=inp1[27:,:,:]
y_train1=out1[:27]
y_test1=out1[27:]    

In [ ]:
predicted=model.fit(x_train1,y_train1,epochs=250)

# the error is showing less because the data is scaled in range(-1,1)

In [ ]:
predict_train=model.predict(x_test1)
plt.figure(figsize=(20,5))
plt.plot(predict_train)
plt.plot(y_test1)
plt.title('Test Data Line vs Predicted Data Line')

In [ ]:
# Suboptimally  performing equipment identification:




In [ ]:
merged=pd.merge(dfGeneration_1,dfWeatherSensor_1,on='DATE')


In [ ]:
merged.set_index(['DATE_TIME_x'],inplace=True)

In [ ]:
plt.plot(merged.groupby(['HOUR_x'])['DC_POWER'].sum())

In [ ]:
part1=dfGeneration_1.loc[dfGeneration_1['SOURCE_KEY']=='1BY6WEcLGh8j5v7']

In [ ]:
merged=pd.merge(part1,dfWeatherSensor_1,on='DATE_TIME',how='left')

In [ ]:
merged['temp_diff']=merged['AMBIENT_TEMPERATURE']-merged['MODULE_TEMPERATURE']

In [ ]:
ready=merged.iloc[:,[2,3,9,13,14,15,18,20]]

In [ ]:
y=ready['DC_POWER']

In [ ]:
x=ready.iloc[:,1:]

In [ ]:
# this model can be further optimized using Random SearchCV, gridSearchCV, (skipping due to shortage of time, you can check hyper paramter optimization
#in my other notebooks)

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=2)
model=xgb.XGBRegressor()
model.fit(x_train,y_train)
aa=model.predict(x_test)

In [ ]:
plt.figure(figsize=(18,8))
sns.distplot(y_train,color='red',label='Predicted value')
sns.distplot(aa,label='Actual value')
plt.legend()

# Thank you